In [2]:
# IMPORTS
import requests
import numpy as np
import time
import pandas as pd
import os
import datetime
from io import TextIOWrapper
from fastparquet import write
from pandas import DataFrame
from riotwatcher import LolWatcher, ApiError
from pynput import keyboard

In [5]:
API_KEY = os.getenv('RIOT_API_KEY')
EXPECTED_SUMMONERS_OUT = 205
API_KEY = 'RGAPI-ae6b149e-f547-4b4d-b86b-7bb45695d6c7'
lol_watcher = LolWatcher(API_KEY)

In [8]:
def get_summoner_puuids(region: str, summonerID: str) -> tuple[DataFrame, bool]:
    """
    Gets random summoner ids from North America from the various ranks (Diamond:Platinum:Gold:Silver:Bronze)
    Returns a pandas dictionary only with summoner ids that have wins + losses >= 20
    Args:
        region (str): The region we want to search in
        rank (str): The rank we want to get from
        division (str): The divions we want to get from
        num_pages (int, optional): An amount of summoner ids to get, a group of 204 are a 'page'. Defaults to 4.

    Returns:
        bool: Whether getting the summoner ids was successful
    """
    print(f"Current summonerID: {summonerID}")

    try:
        # attempt to get a response
        response = lol_watcher.summoner.by_id(region, summonerID)

    except ApiError as err:
        print(err)
        print(f'Failed at summonerID: {summonerID}')
        return None, False

    return pd.DataFrame.from_records(response, index=[0]), True

In [10]:
def write_to_parquet(df: DataFrame, region: str, rank: str, tier: str) -> str:
    DIRECTORY_PATH = f'F://data//puuids//{region}//'
    date_str = str(datetime.datetime.today()).split()[0]
    file_name = f'puuids_{rank}_{tier}.parquet'
    file_path = DIRECTORY_PATH + file_name
    # create the parquet if it doesn't exist
    if not os.path.isfile(file_path):
        print(f'Creating {file_name}')
        write(file_path, df)
    else:
        print(f'Appending to {file_name}')
        write(file_path, df, append=True)
    return file_path

In [3]:
test = pd.read_parquet('C://Repository//MOBA_Project//data//summoner//na1//summoner_data_PLATINUM_I.parquet')

In [6]:
test['summonerId'].iloc[12100:]

12100    fe3AN-MZTo9rFpSovOR7msvvHqgnckvd7jSOjKYpoClubpw
12101    gUiBY3NOeUA_FlxFTutA2tu7jT3sJL1pWPj6UMy4k-ANCBk
12102    UDN29yPqJcQVNBg6Fs22BdJvr5tOtIw4EXSiQQ_kmMBhwlo
12103    Dsgfz3T-kRVQspfuiWEb6f8y7w7S9Nw1Ybnehbdl_tUAr7Y
12104    8ywwxvzKD8zaAR8ePrYdCdkxpR4A1hLL2xu6yxpR7lJ2BMw
                              ...                       
17464    EFv6XxkOmmIgyriF9Nx30PQNsCpmELToyAqoxTc8huCq_yQ
17465    pKfRPKK3J_UdETgwLm7j3YBfENbGzIKGEXhtLyRF49RhCgY
17466    nK59vjh6Eq79INAvsWLUkqWaxuKKbdvOlxF_r4xWF_yQ56A
17467    baH1XiOtBn9de6uGR6F2-6hMQJDQQCObI2BEKwqaVeZT5Ps
17468    1soFewjd51FJx_M7SMFIqkJfQPb7fyrW5QiR5wij4j7Yrbo
Name: summonerId, Length: 5369, dtype: object

In [11]:
regions = ['na1']
ranks = ['PLATINUM']
tiers = ['I', 'II', 'III', 'IV']
SUMMONER_DATA_FP = 'C://Repository//MOBA_Project//data//summoner//'
s = []
stop_flag = False
# Read the summonerIDs for the region
for region in regions:
    for rank in ranks:
        for tier in tiers:
            # read the summoner dataframe for the given region
            file_name = f'{region}//summoner_data_{rank}_{tier}.parquet'
            complete_fp = SUMMONER_DATA_FP + file_name

            summoners = pd.read_parquet(complete_fp, columns=['summonerId'])
            seen_summoners = set()
            for summoner in summoners['summonerId']:
                
                if summoner in seen_summoners:
                    continue

                seen_summoners.add(summoner)
                df, success = get_summoner_puuids(region, summoner)

                if not success:
                    stop_flag = True

                write_to_parquet(df, region, rank, tier)

Current summonerID: Ds6ZpIEpbAEbJWgWAWA_-MBaC0_gx6udEOF7k3FXtFWtl20
Creating puuids_PLATINUM_I.parquet
Current summonerID: Bk0zAl4opeSuAxIKR2-szw6ujiWPjoyfE3ynULcN9cP9Lek
Appending to puuids_PLATINUM_I.parquet
Current summonerID: 2lGnnh_0v-K3lWX-sTpf5C8OEDyWtm-itBXFlj626w_Ccv8
Appending to puuids_PLATINUM_I.parquet
Current summonerID: bhRVDKPkm2YZ03x2XE1qAAjj9twSUyM5jzBtSU5lnVE1h1Q
Appending to puuids_PLATINUM_I.parquet
Current summonerID: ZAQ6HMO9wHJBGBlJmg4PAJ-FWTvA0takisZK6bXjBnMuJn8
Appending to puuids_PLATINUM_I.parquet
Current summonerID: MfsXpLPBDpNkCH9Uug086L7UjLT03Byq1VKIOOro7oOJW-Y
Appending to puuids_PLATINUM_I.parquet
Current summonerID: FgB_3-NKfNj9YVkjHhG8ft-55OuL6Ah0ugnfnZ69fvBttA0
Appending to puuids_PLATINUM_I.parquet
Current summonerID: VUDkRiBueKqxeJmvTVTc_m72lxU-iX8TXoJ20PS64FE2064
Appending to puuids_PLATINUM_I.parquet
Current summonerID: uxp5IZS-76Qqk3cPe_6SYC1F_0yO8IvHUtQot3DiBmOZMKo
Appending to puuids_PLATINUM_I.parquet
Current summonerID: bLep3HgKfyYqith9Z_eB0